<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 15px; height: 80px">

# Project 2

### Exploratory Data Analysis (EDA)

---

Your hometown mayor just created a new data analysis team to give policy advice, and the administration recruited _you_ via LinkedIn to join it. Unfortunately, due to budget constraints, for now the "team" is just you...

The mayor wants to start a new initiative to move the needle on one of two separate issues: high school education outcomes, or drug abuse in the community.

Also unfortunately, that is the entirety of what you've been told. And the mayor just went on a lobbyist-funded fact-finding trip in the Bahamas. In the meantime, you got your hands on two national datasets: one on SAT scores by state, and one on drug use by age. Start exploring these to look for useful patterns and possible hypotheses!

---

This project is focused on exploratory data analysis, aka "EDA". EDA is an essential part of the data science analysis pipeline. Failure to perform EDA before modeling is almost guaranteed to lead to bad models and faulty conclusions. What you do in this project are good practices for all projects going forward, especially those after this course!

This lab includes a variety of plotting problems. Much of the plotting code will be left up to you to find either in the lesson notes, or if not there, online. There are massive amounts of code snippets either in documentation or sites like [Stack Overflow](https://stackoverflow.com/search?q=%5Bpython%5D+seaborn) that have almost certainly done what you are trying to do.

**Get used to googling for code!** You will use it every single day as a data scientist, especially for visualization and plotting.

#### Package imports

In [ ]:
import numpy as np
import scipy.stats as stats
import csv
import pandas as pd
from pprint import pprint
import matplotlib.pyplot as plt

plt.style.use('ggplot')
plt.rcParams['font.size'] = 14

# this line tells jupyter notebook to put the plots in the notebook rather than saving them to file
%matplotlib inline

# this line makes plots prettier on mac retina screens
%config InlineBackend.figure_format = 'retina'

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 1. Load the `sat_scores.csv` dataset and describe it

---

You should replace the placeholder path to the `sat_scores.csv` dataset below with your specific path to the file.

### 1.1 Load the file with the `csv` module and put it in a Python dictionary

The dictionary format for data will be the column names as key, and the data under each column as the values.

Toy example:
```python
data = {
    'column1':[0,1,2,3],
    'column2':['a','b','c','d']
    }
```

In [ ]:
def convert(x):
    if x.isnumeric():
        return int(x)
    else:
        return str(x)

In [ ]:
import csv

data = {}
with open('sat_scores.csv', newline='') as sat_file:
    reader = csv.DictReader(sat_file)
    for i, row in enumerate(reader):
        for col in row.keys():
            if i == 0:
                data[col] = [convert(row[col])]
            else:
                data[col].append(convert(row[col]))
       

In [ ]:
# print first 5 entries of each key
pprint({k: data[k][:5] for k in data})

### 1.2 Make a pandas DataFrame object with the SAT dictionary, and another with the pandas `.read_csv()` function

Compare the DataFrames using the `.dtypes` attribute in the DataFrame objects. What is the difference between loading from file and inputting this dictionary (if any)?

In [ ]:
sat_scores_from_dict = pd.DataFrame(data)

In [ ]:
sat_scores = pd.read_csv('sat_scores.csv')

In [ ]:
print(sat_scores_from_dict.dtypes)

In [ ]:
print(sat_scores.dtypes)

If you did not convert the string column values to float in your dictionary, the columns in the DataFrame are of type `object` (which are string values, essentially). 

### 1.3 Look at the first ten rows of the DataFrame: what does our data describe?

From now on, use the DataFrame loaded from the file using the `.read_csv()` function.

Use the `.head(num)` built-in DataFrame function, where `num` is the number of rows to print out.

You are not given a "codebook" with this data, so you will have to make some (very minor) inference.

In [ ]:
sat_scores.head(10)

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 2. Create a "data dictionary" based on the data

---

A data dictionary is an object that describes your data. This should contain the name of each variable (column), the type of the variable, your description of what the variable is, and the shape (rows and columns) of the entire dataset.

In [ ]:
feature_names = list(sat_scores.columns)
feature_names

In [ ]:
# data = [list(sat_scores[col]) for col in sat_scores.columns]
data = np.array(sat_scores)
data[:5]

In [ ]:
description = """SAT Scores (standardized college admissions test):

:Description: The score range for the SAT is 400-1600. It's the sum on the 
              Evidence-Based Reading and Writing (ERW) section and the Math section, 
              which each have a score range of 200-800.
:Number of Instances: {instance}
:Number of Attributes: {attr}
:Attribute Information (in order):
    - State     US states (abbreviations)
    - Rate      Rate in previous school (1 is the best)
    - Verbal    Verbal score
    - Math      Math score
""".format(instance=len(data[0]), attr=len(data))

In [ ]:
print(description)

In [ ]:
import os

# filename = os.path.dirname(os.getcwd()) + '/EDA/sat_scores.csv'
filename = 'https://github.com/androi7/ds-mini-projects/tree/main/EDA/sat_scores.csv'

In [ ]:
sat_scores = {'data': data, 'description': description, 'feature_names': feature_names, 'filename': filename}

In [ ]:
# print only the first five rows of the data key
pprint({k: sat_scores[k][:5] if k == 'data' else sat_scores[k] for k in sat_scores})

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 3. Plot the data using seaborn

---

### 3.1 Using seaborn's `distplot`, plot the distributions for each of `Rate`, `Math`, and `Verbal`

Set the keyword argument `kde=False`. This way you can actually see the counts within bins. You can adjust the number of bins to your liking. 

[Please read over the `distplot` documentation to learn about the arguments and fine-tune your chart if you want.](https://stanford.edu/~mwaskom/software/seaborn/generated/seaborn.distplot.html#seaborn.distplot)

In [ ]:
sat_scores = pd.DataFrame(data=sat_scores['data'], columns=sat_scores['feature_names'])
sat_scores.columns

In [ ]:
sat_scores.info()

In [ ]:
# def convert(x, dtype):
#     try:
#         return dtype(x)
#     except:
#         return np.nan

In [ ]:
sat_scores = sat_scores.astype({'Rate': 'int32', 'Verbal': 'int32', 'Math': 'int32'})
sat_scores.dtypes

In [ ]:
import seaborn as sns
sns.set(font_scale=1.5)
plt.style.use('seaborn')
plt.rcParams['axes.labelsize'] = 14

fig, ax = plt.subplots(ncols=3, figsize=(12, 4))

# distplot is deprecated soon
g = sns.histplot(sat_scores.Rate, kde=False, bins=10, ax=ax[0])
ax[0].set_title('Rate distribution\n', fontsize=18)
ax[0].set_ylabel('Counts')
sns.histplot(sat_scores.Verbal, kde=False, bins=10, ax=ax[1])
ax[1].set_title('Verbal distribution\n', fontsize=18)
ax[1].set_ylabel('Counts')
sns.histplot(sat_scores.Math, kde=False, bins=10, ax=ax[2])
ax[2].set_title('Math distribution\n', fontsize=18)
ax[2].set_ylabel('Counts')
fig.tight_layout(pad=1)
plt.show()

### 3.2 Using seaborn's `pairplot`, show the joint distributions for each of `Rate`, `Math`, and `Verbal`

Explain what the visualization tells you about your data.

[Please read over the `pairplot` documentation to fine-tune your chart.](https://stanford.edu/~mwaskom/software/seaborn/generated/seaborn.pairplot.html#seaborn.pairplot)

In [ ]:
g = sns.pairplot(sat_scores, corner=True, diag_kws={'bins': 10, 'color': 'dodgerblue'})
g.fig.suptitle('Pairplot of the SAT results', y=1.08)
plt.show()

The pairplot depicts a histogram plot on the diagonal (univariate distribution) where the variables on the x- and y-axis are the same and it shows a scatter plot where the variables are different.

Furthermore, the *Verbal* assignment is strongly negative correlated to the SAT *Rate*, because the *Verbal* is also strongly positive correlated to the *Math* score. Which means that students who gain good results in the reading and writing section also have a well understanding in math. The same relation applies to the under end of the score range and everything in between.

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 4. Plot the data using built-in pandas functions.

---

Pandas is very powerful and contains a variety of nice, built-in plotting functions for your data. Read the documentation here to understand the capabilities:

http://pandas.pydata.org/pandas-docs/stable/visualization.html

### 4.1 Plot a stacked histogram with `Verbal` and `Math` using pandas

In [ ]:
from matplotlib.colors import ListedColormap
colors = ['dodgerblue', 'orange']

sat_scores[['Math', 'Verbal']].plot.hist(stacked=True, bins=10, colormap=ListedColormap(colors))
plt.title('Histogram of Verbal and Math\n', fontsize=18)
plt.show()

### 4.2 Plot `Verbal` and `Math` on the same chart using boxplots

What are the benefits of using a boxplot as compared to a scatterplot or a histogram?

What's wrong with plotting a box-plot of `Rate` on the same chart as `Math` and `Verbal`?

In [ ]:
ax = sat_scores.boxplot(column=['Math', 'Verbal'], figsize=(6,4), color={'boxes':'Black', 'medians': 'Red'}, 
                        medianprops={'linestyle': '--', 'linewidth': 2}, return_type='dict', 
                        patch_artist=True, fontsize=14)

for i, box in enumerate(ax['boxes']):
    box.set_facecolor(colors[i])
plt.title('Boxplot of Math and Verbal\n', fontsize=18)
plt.show()

Boxplots are primarly used to compare several distributions against each other. They show easily the five number summary (Min, Q1, Median, Q3, Max). Compared to histograms you can extract the exact positions of the outliers and you see the median and the interquartile range by default. And it's easier to determine the skewness. 
When using the scatterplot instead, you would neither detect the median, IQR, nor the outliers.

The Rate column has different dimensions and it would squeeze the plot if the data aren't normalized.

<img src="http://imgur.com/xDpSobf.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### 4.3 Plot `Verbal`, `Math`, and `Rate` appropriately on the same boxplot chart

Think about how you might change the variables so that they would make sense on the same chart. Explain your rationale for the choices on the chart. You should strive to make the chart as intuitive as possible. 


In [ ]:
def standarized(x):
    "Returns the z-score"
    return (x-x.mean())/x.std()

In [ ]:
sat_scores_normalized = sat_scores[['Rate', 'Verbal', 'Math']].apply(standarized)
ax = sat_scores_normalized.boxplot(column=['Math', 'Verbal', 'Rate'], 
                                   figsize=(6,4), 
                                   color={'boxes':'Black', 'medians': 'Red'}, 
                                   medianprops={'linestyle': '--', 'linewidth': 2},
                                   return_type='dict',
                                   patch_artist=True,
                                   fontsize=14)

colors.append('turquoise')
for i, box in enumerate(ax['boxes']):
    box.set_facecolor(colors[i])
    
plt.title('Boxplot of standardized Math, Verbal and Rate\n', fontsize=18)
plt.show()

Each column of the dataset was normalised by calculating the z-scores. While each column has now a distribution with the mean of null and a standard deviation of one, they can be used for comparisons.

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 5. Create and examine subsets of the data

---

For these questions you will practice **masking** in pandas. Masking uses conditional statements to select portions of your DataFrame (through boolean operations under the hood.)

Remember the distinction between DataFrame indexing functions in pandas:

    .iloc[row, col] : row and column are specified by index, which are integers
    .loc[row, col]  : row and column are specified by string "labels" (boolean arrays are allowed; useful for rows)
    .ix[row, col]   : row and column indexers can be a mix of labels and integer indices
    
For detailed reference and tutorial make sure to read over the pandas documentation:

http://pandas.pydata.org/pandas-docs/stable/indexing.html



### 5.1 Find the list of states that have `Verbal` scores greater than the average of `Verbal` scores across states

How many states are above the mean? What does this tell you about the distribution of `Verbal` scores?




In [ ]:
verbal_mean = sat_scores['Verbal'].mean()
print('Verbal mean:', round(verbal_mean))
mask = (sat_scores['Verbal'] >  verbal_mean)
# alphabetically ordered
sat_scores_states_above_verbal_mean = sat_scores[mask][['State', 'Verbal']].sort_values(by=['State'])
sat_scores_states_above_verbal_mean.head()

In [ ]:
print("%s states have a higher Verbal score than the mean." % sat_scores_states_above_verbal_mean.shape[0])

In [ ]:
print('Counts of states:', sat_scores.shape[0])
verbal_median = sat_scores['Verbal'].median()
print('Verbal median:', verbal_median)
print('Verbal distribution skewness:', round(sat_scores[['Verbal']].skew()[0], 4))

In [ ]:
sat_scores['Verbal'].hist(bins=15)
plt.title('Histogram of Verbal\n', fontsize=18)
plt.show()

On half of the states is above and other half is under the mean of the Verbal score. The distribution is positive skewed and multimodal (probably bimodal based on the given plot with 15 bins). 

### 5.2 Find the list of states that have `Verbal` scores greater than the median of `Verbal` scores across states

How does this compare to the list of states greater than the mean of `Verbal` scores? Why?

In [ ]:
mask = (sat_scores['Verbal'] > verbal_median)
sat_scores_states_above_verbal_median = sat_scores[mask][['State', 'Verbal']].sort_values(by=['State'])
sat_scores_states_above_verbal_median.head()

In [ ]:
print("%s states are above the median Verbal score." % sat_scores_states_above_verbal_median.shape[0])

For positive skewed distributions the mean value is greater than the median value.

### 5.3 Create a column that is the difference between the `Verbal` and `Math` scores

Specifically, this should be `Verbal - Math`.

In [ ]:
sat_scores['Verbal_Math_diff'] = sat_scores['Verbal'] - sat_scores['Math']
sat_scores.head()

### 5.4 Create two new DataFrames showing states with the greatest difference between scores

1. Your first DataFrame should be the 10 states with the greatest gap between `Verbal` and `Math` scores where `Verbal` is greater than `Math`. It should be sorted appropriately to show the ranking of states.
2. Your second DataFrame will be the inverse: states with the greatest gap between `Verbal` and `Math` such that `Math` is greater than `Verbal`. Again, this should be sorted appropriately to show rank.
3. Print the header of both variables, only showing the top 3 states in each.

In [ ]:
mask = (sat_scores['Verbal'] > sat_scores['Math'])  # to be sure that verbal score is greater 
sat_scores_verbal_greater_math = sat_scores[mask].drop(labels=['Rate'], axis=1) \
                                     .sort_values(by=['Verbal_Math_diff'], ascending=False)[:10]
sat_scores_verbal_greater_math

In [ ]:
mask = (sat_scores['Verbal'] < sat_scores['Math'])  # to be sure that math score is greater 
sat_scores_math_greater_verbal = sat_scores[mask].drop(labels=['Rate'], axis=1) \
                                     .sort_values(by=['Verbal_Math_diff'], ascending=True)[0:10]
sat_scores_math_greater_verbal

In [ ]:
sat_scores_verbal_greater_math.head(3)

In [ ]:
sat_scores_math_greater_verbal.head(3)

## 6. Examine summary statistics

---

Checking the summary statistics for data is an essential step in the EDA process!

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### 6.1 Create the correlation matrix of your variables (excluding `State`).

What does the correlation matrix tell you?


In [ ]:
sat_scores.drop('Verbal_minus_Math', axis=1, inplace=True)

In [ ]:
sat_scores[['Rate', 'Verbal', 'Math']].corr()

Rate and Verbal are strong negatively correlated which means that one variable declines while the other increases. Rate and Math are a little be lower negatively correlated than the previous variables. Math and Verbal are strong positively related which means that both either increase or decrease together.

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### 6.2 Use pandas'  `.describe()` built-in function on your DataFrame

Write up what each of the rows returned by the function indicate.

In [ ]:
sat_scores.describe()

For columns with numerical values:<br>
<b>count</b>: amount of variables<br>
<b>mean</b>: the average value<br>
<b>std</b>: the standard deviation<br>
<b>min</b>: the minimum value<br>
<b>25%</b>: 25th percentile (25% of the values are equal or lower than this value)<br>
<b>50%</b>: median value<br>
<b>75%</b>: 75th percentile<br>
<b>max</b>: the maximum value

In [ ]:
sat_scores.describe(include='O')

For columns with non-numerical values:<br>
<b>count</b>: amount of variables<br>
<b>unique</b>: amount of unique values<br>
<b>top</b>: value which occures the most often<br>
<b>freq</b>: how many times the <b>top</b> value occures

<img src="http://imgur.com/xDpSobf.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### 6.3 Assign and print the _covariance_ matrix for the dataset

1. Describe how the covariance matrix is different from the correlation matrix.
2. What is the process to convert the covariance into the correlation?
3. Why is the correlation matrix preferred to the covariance matrix for examining relationships in your data?

In [ ]:
sat_scores[['Rate', 'Verbal', 'Math']].cov()

$$ \text{covariance}(X, Y) = \sum_{i=1}^N \frac{(X_i - \bar{X})(Y_i - \bar{Y})}{N-1}$$

$$ \text{Pearson correlation}\;r = {\rm corr}(X, Y) =\frac{{\rm cov}(X, Y)}{{\rm std}(X){\rm std}(Y)}$$

1. Both measure the relationship between two variables, whereas the covariance only indicates the direction the correlation additionaly shows the strength of the direction. But the correlation matrix is normalized by the standard deviations of the two variables.

2. To convert the covariance matrix into a correlation matrix it has to be normalized by the two standard deviations of both variables. And the end the values are in the interval of [-1, 1]. 
3. With the correlation matrix we can also determine the strength of the relationship of two variables next to the direction of it. It is also a unit-free masure whereas the covariance matrix has the unit of the product of the two variables. The range of [-1, 1] lets us easily detect the strength of the relationship.

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 7. Performing EDA on "drug use by age" data.

---

You will now switch datasets to one with many more variables. This section of the project is more open-ended - use the techniques you practiced above!

We'll work with the "drug-use-by-age.csv" data, sourced from and described here: https://github.com/fivethirtyeight/data/tree/master/drug-use-by-age.

### 7.1 Load the data using pandas. 

Does this data require cleaning? Are variables missing? How will this affect your approach to EDA on the data?

In [ ]:
drug_use_by_age_csv = "./drug-use-by-age.csv"  # os.path.dirname(os.getcwd()) + "/EDA/drug-use-by-age.csv"
drug = pd.read_csv(drug_use_by_age_csv)

In [ ]:
drug.shape

In [ ]:
drug.columns

In [ ]:
import string

drug.columns = [('').join([letter.lower() 
                           if letter in string.ascii_letters 
                           else '_' if i!=0 and not i == len(col)-1 
                           else '' for letter in col]) 
                for i, col in enumerate(drug.columns)]
drug.columns

In [ ]:
drug.head()

In [ ]:
drug.tail()

In [ ]:
drug.info()

In [ ]:
drug.iloc[:, :15].describe()

In [ ]:
drug.iloc[:,15:].describe()

In [ ]:
stimulant_frequency_75perc = np.quantile(drug['stimulant_frequency'], 0.75)
print(stimulant_frequency_75perc)
drug[drug['stimulant_frequency'] > stimulant_frequency_75perc][['stimulant_frequency']]

In [ ]:
# drug.loc[16, 'stimulant_frequency'] = 34.0
drug.tail(1)

In [ ]:
drug.drop(drug.index[-1], inplace=True)

In [ ]:
drug.describe(include='O')

In [ ]:
for col in drug.describe(include='O').columns:
    print("""
    Column name: {col}
    Unique values: {val}
    """.format(col=col, val=drug[col].unique()))

In [ ]:
def convert_to_float(x):
    "Return value converted to numeric float value or nan"
    try:
        return float(x)
    except:
        return np.nan

In [ ]:
for i, col in enumerate(drug.describe(include='O').columns):
    if i > 0:
        drug[col] = drug[col].map(convert_to_float)

In [ ]:
drug.dtypes

In [ ]:
drug.isnull().sum()

### 7.2 Do a high-level, initial overview of the data

Get a feel for what this dataset is all about.

Use whichever techniques you'd like, including those from the SAT dataset EDA. The final response to this question should be a written description of what you infer about the dataset.

Some things to consider doing:

- Look for relationships between variables and subsets of those variables' values
- Derive new features from the ones available to help your analysis
- Visualize everything!

In [ ]:
plt.figure(figsize=(30,30))
sns.heatmap(drug.corr(), annot=True)
plt.show()

In [ ]:
# age as categorical variable and to be used in the correlation matrix to indicate relations
drug['age_labelled'] = drug['age'].apply(lambda x: drug[drug['age'] == x].index[0] + 1)
drug.head()

In [ ]:
# highest correlations
plt.figure(figsize=(30,30))
sns.heatmap(drug.corr()[(drug.corr() > 0.8) | (drug.corr() < -0.8)] , annot=True)
plt.show()

In [ ]:
print(list(zip(drug['age'], drug['age_labelled'])))

In [ ]:
ax = drug.plot('age_labelled', ['crackuse', 'inhalant_use'], style='.-')
ax.set_title('Drug use in percentage per age groups\n');

In [ ]:
ax = drug.plot('age_labelled', 'alcohol_use', style='.-')
ax.set_title('Alcohol use in percentage per age groups\n');

In [ ]:
ax = drug.plot('age_labelled', ['marijuana_frequency', 'alcohol_frequency'], style='.-')
ax.set_title('Frequency per age groups\n');

Inference:<br>
The drugs with the highest correlation to the age of the candidates are marijuna, alcohol, crack and inhalants. Whereas the consumption of each drug of that list increases with the age only the inhalants have their peak in very early age and decrease while getting older. The other drugs have their peak more or less around the age of 21.

### 7.3 Create a testable hypothesis about this data

Requirements for the question:

1. Write a specific question you would like to answer with the data (that can be accomplished with EDA).
2. Write a description of the "deliverables": what will you report after testing/examining your hypothesis?
3. Use EDA techniques of your choice, numeric and/or visual, to look into your question.
4. Write up your report on what you have found regarding the hypothesis about the data you came up with.


Your hypothesis could be on:

- Difference of group means
- Correlations between variables
- Anything else you think is interesting, testable, and meaningful!

**Important notes:**

You should be only doing EDA _relevant to your question_ here. It is easy to go down rabbit holes trying to look at every facet of your data, and so we want you to get in the practice of specifying a hypothesis you are interested in first and scoping your work to specifically answer that question.

Some of you may want to jump ahead to "modeling" data to answer your question. This is a topic addressed in the next project and **you should not do this for this project.** We specifically want you to not do modeling to emphasize the importance of performing EDA _before_ you jump to statistical analysis.

** Question and deliverables**


...

1) Null Hypothesis: Drug consumption is not related to the age of the candidates.

2) Calculating the correlation matrix and detect the direction and strength of the relationships of the individual drugs and the age parameter. After gaining the result try to state inferences why the relationship is given or not. If the null hypothesis is wrong try to divide the ages in useful groups and find the causality for the impact. What are the differences between these groups which could lead to the given result.

3) See result above (also inference). 

In [ ]:
# Code

**Report**



...

<img src="http://imgur.com/xDpSobf.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 8. Introduction to dealing with outliers

---

Outliers are an interesting problem in statistics, in that there is not an agreed upon best way to define them. Subjectivity in selecting and analyzing data is a problem that will recur throughout the course.

1. Pull out the rate variable from the sat dataset.
2. Are there outliers in the dataset? Define, in words, how you _numerically define outliers._
3. Print out the outliers in the dataset.
4. Remove the outliers from the dataset.
5. Compare the mean, median, and standard deviation of the "cleaned" data without outliers to the original. What is different about them and why?

In [ ]:
def find_outliers(df, col):
    """Return and print outliers
    
    Parameters:
    df (DataFrame): dataframe
    col (str): name of the column
    
    Returns:
    lower_outlier (pandas.Series): list of all lower outliers
    upper_outlier (pandas.Series): list of all upper outliers
    """
    mean_val = df[col].mean()
    std_val = df[col].std()
    
    mask_lower = (df[col] < mean_val - 3 * std_val)
    mask_upper = (df[col] > mean_val + 3 * std_val)
    
    lower_outlier = df[col][mask_lower]
    upper_outlier = df[col][mask_upper]
    print("In {col} are {amount} lower outliers: {lower_outliers}".format(col=col, amount=len(lower_outlier), lower_outliers=[v for v in lower_outlier]))
    print("In {col} are {amount} upper outliers: {upper_outliers}".format(col=col, amount=len(upper_outlier), upper_outliers=[v for v in upper_outlier]))
    
    return lower_outlier, upper_outlier

In [ ]:
find_outliers(sat_scores, 'Rate')

2)<br>
1. Method
$$outlier_{lower}=Q_{1}-1.5*IQR$$
$$outlier_{upper}=Q_{3}-1.5*IQR$$
2. Method
$$outlier_{lower}=mean-3*std$$
$$outlier_{upper}=mean+3*std$$

In [ ]:
sat_scores = pd.read_csv('sat_scores.csv')
sat_scores_normalized = sat_scores[['Rate', 'Verbal', 'Math']].apply(standarized)
sat_scores_normalized.boxplot(column=['Math', 'Verbal', 'Rate'], figsize=(6,4), color={'boxes':'Blue', 'medians': 'Red'}, medianprops={'linestyle': '--', 'linewidth': 2});

No outliers in the sat_score dataframe.

In [ ]:
sat_scores['Math'].sort_values()[27]

In [ ]:
mean_val = sat_scores['Math'].mean()
std_val = sat_scores['Math'].std()
median_val = sat_scores['Math'].median()
    
z_value = (sat_scores.loc[27, 'Math'] - mean_val)/std_val
z_value
# This value is not by definition an outlier, but the z-score is close to the limit of it.

In [ ]:
sat_scores.drop(sat_scores.index[27], inplace=True)

In [ ]:
mean_val_cleaned = sat_scores['Math'].mean()
std_val_cleaned = sat_scores['Math'].std()
median_val_cleaned = sat_scores['Math'].median()

print(f'Median - before: {median_val} | cleaned: {median_val_cleaned}')
print(f'Standard deviation - before: {round(std_val, 2)} | cleaned: {round(std_val_cleaned, 2)}')
print(f'Mean - before: {mean_val} | cleaned: {round(mean_val_cleaned, 2)}')

1) The median value takes the next greater value (if length of the sequence is odd, else the average), because one outlier was dropped.
2) Standard deviation is smaller when the dataframe is cleaned, because outliers stretch the range.
3) The outlier was a lower outlier and therefore the mean value increases after cleaning the dataframe, because a smaller value was dropped.

<img src="http://imgur.com/GCAf1UX.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### 9. Percentile scoring and Spearman rank correlation

---

### 9.1 Calculate the Spearman correlation of sat `Verbal` and `Math`

1. How does the Spearman correlation compare to the Pearson correlation? 
2. Describe clearly in words the process of calculating the Spearman rank correlation.
  - Hint: the word "rank" is in the name of the process for a reason!


$$ \text{Spearman correlation}\;r = 1-\frac{6\sum_{i}d^{2}_{i}}{n(n^2-1)}$$

d... difference of the ranks of two variables for each row<br>
n... amount of numbers per column

rank: labelling the values with numbers (ranks) from 1 for the highest value in ordered sequence up to the highest rank for the smallest value

The Spearman correlation can therefore also be used to evaluate relationships involving ordinal (categorical) variables.

The strengs of the relationships are less with the Spearman correlation formula.

In [ ]:
sat_scores.corr(method='spearman')

$$ \text{Pearson correlation}\;r = {\rm corr}(X, Y) =\frac{{\rm cov}(X, Y)}{{\rm std}(X){\rm std}(Y)}$$

In [ ]:
sat_scores.corr(method='pearson')

### 9.2 Percentile scoring

Look up percentile scoring of data. In other words, the conversion of numeric data to their equivalent percentile scores.

http://docs.scipy.org/doc/numpy-dev/reference/generated/numpy.percentile.html

http://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.percentileofscore.html

1. Convert `Rate` to percentiles in the sat scores as a new column.
2. Show the percentile of California in `Rate`.
3. How is percentile related to the Spearman rank correlation?

In [ ]:
from scipy import stats

sat_scores['Rate_Percentile'] = sat_scores['Rate'].apply(lambda x: stats.percentileofscore(sat_scores['Rate'], x))
sat_scores.head()

In [ ]:
sat_scores[sat_scores['State'] == 'CA'][['State', 'Rate_Percentile']]

Percentile is related to the rank in the manner of that the values are sorted and allocated to a new value based on their position of the ordered sequence. Rank as label from 1 for the highest value up to a number for the lowest value. The percentile score is set to 100 (percent) for the highest value up to a proportion of percentage for the lowest value.

### 9.3 Percentiles and outliers

1. Why might percentile scoring be useful for dealing with outliers?
2. Plot the distribution of a variable of your choice from the drug use dataset.
3. Plot the same variable but percentile scored.
4. Describe the effect, visually, of converting raw scores to percentile.

With the 25th and the 75th percentile the interquartile range can be calculated which is used to detect outliers (see formulas above).

In [ ]:
ages = [age for age in drug['age']]




fig, ax = plt.subplots(figsize=(6,4))
drug['alcohol_use'].hist(ax=ax);
ax.set_title('Alcohol use distribution\n')
ax.set_xlabel('Alcohol use percentage per age group')
ax.set_ylabel('Counts');

#print(ages)

In [ ]:
drug['alcohol_use_percentile'] = drug['alcohol_use'].apply(lambda x: stats.percentileofscore(drug['alcohol_use'], x))

fig, ax = plt.subplots(figsize=(6,4))
drug['alcohol_use_percentile'].hist(ax=ax);
ax.set_title('Alcohol use distribution in percentiles\n')
ax.set_xlabel('Alcohol use percentiles per age group')
ax.set_ylabel('Counts');

In [ ]:
drug[['alcohol_use', 'alcohol_use_percentile']].sort_values(by=['alcohol_use'], ascending=False)

Percentile score:<br>
lowest column value allocated with $$i=100/len(column)$$ which is the step variable for each next higher value in the column in an ordered sequence.

In [ ]:
drug[['alcohol_use', 'age']]